# 모듈 만들기 (Building a Module)

파이썬에서 모듈이란 개념적으로 모듈 임포트를 통해 접근가능하고 구동가능한 파이썬 코드들이 들어있는 파일입니다.

모듈을 `import`하면, 파일을 읽고 파일에 포함된 코드를 별칭을 통해 액세스할 수 있게 됩니다.

우선 `hello.py`라는 새 파일을 만들고 거기에 아래와 같은 메시지를 추가해보겠습니다.

```python
message = 'Hello Jim'
```

다음으로는 코드를 실행해 보겠습니다.

```python
import hello as h
h.message
```

좋습니다!

이제 메시지를 변경해 보겠습니다. 이제 다음과 같이 말하도록 파일을 편집합니다.

```python
message = 'Hello Jane'
```

코드를 다시 실행해보겠습니다.

```python
import hello as h
h.message
```

어라? 뭐가 잘못되었을까요?

모듈의 내용은 모듈이 개발 중인 상황을 제외하고는 일반적으로 자주 변경되지 않기 때문에, 똑똑한 우리의 파이썬은 모듈을 한번 가져오면 모듈을 다시 가져오는 데 신경을 쓰지 않습니다. 일반적으로는 이것이 우리가 원하는 것입니다. 하지만 여기서는 실습의 목적상 모듈의 내용이 변경될 때마다 자동으로 다시 로드되도록 강제하고 싶습니다.

다행히도 정확히 그렇게 만들어주는 **마법의** 명령 시퀀스가 있습니다. 먼저 마법의 구문인 `%load_ext autoreload`를 실행하여 `autoreload`라는 확장 프로그램을 로드해야 합니다. 해당 확장을 로드하면 다양한 자동 재로드 모드를 지원하는 `%autoreload`라는 새로운 명령문에 액세스할 수 있습니다. 우리가 지금 원하는 모드는 변경 사항을 자동으로 다시 로드하는 모드 2입니다. 따라서 다음을 실행해야 합니다.

```python
%load_ext autoreload
%autoreload 2
```

이제 다시 한 번 실행해보겠습니다.

```python
h.message
```

이제 hello 파일에 대한 향후 편집 내용이 즉시 다시 로드되는 것을 볼 수 있습니다.

`hello.py` 파일을 다음과 같이 편집해보겠습니다.

```python
message = 'Hello John and Jane'
```

그 다음 다시 실행을 하면

```python
h.message
```

이제 전체 실습 과정을 통해 빌드할 새로운 모듈을 만든 후에, 이전 실습 시간에 제작한 `drawdown` 함수 및 포트폴리오의 월별 수익률 데이터를 자동으로 가져오는 `get_ffme_returns` 함수를 넣어 보겠습니다. 

함수
1. `drawdown()`
2. `get_ffme_returns()`

`port_opt_toolkit.py`라는 파일을 만들고 다음 코드를 복사합니다.

```python

import pandas as pd

def drawdown(return_series: pd.Series):
    """Takes a time series of asset returns.
       returns a DataFrame with columns for
       the wealth index, 
       the previous peaks, and 
       the percentage drawdown
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, 
                         "Previous Peak": previous_peaks, 
                         "Drawdown": drawdowns})

def get_ffme_returns():
    """
    Load the Fama-French Dataset for the returns of the Top and Bottom Deciles by MarketCap
    """
    me_m = pd.read_csv("data/Portfolios_Formed_on_ME_monthly_EW.csv",
                       header=0, index_col=0, na_values=-99.99)
    rets = me_m[['Lo 10', 'Hi 10']]
    rets.columns = ['SmallCap', 'LargeCap']
    rets = rets/100
    rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')
    return rets

```
